<a href="https://colab.research.google.com/github/Maks2811/test_nagaev/blob/main/task_14_10_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=19f63a4635bfb13dc176d08018c5d23e455b73d8f4d6918239ed8cb0b886ba5e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, sum, mean, count, year, avg

spark = SparkSession.builder \
.appName("book_analytics") \
.getOrCreate()

# создаем датафреймы из csv файлов
#df_book=spark.read.csv("/content/books.csv", header=True, inferSchema=True)
#df_auth=spark.read.csv("/content/authors.csv", header=True, inferSchema=True)
df_book=spark.read.option("header","true").csv("/content/books.csv")
df_auth=spark.read.option("header","true").csv("/content/authors.csv")

#переименовываем колонку author_id во втором датафрейме, чтобы не было дублирующихся названий
df_auth = df_auth.withColumnRenamed("author_id", "auth_id")

# Выводим схемы и сами датафреймы
df_book.printSchema()
df_auth.printSchema()

#df_book.show()
#df_auth.show()

# преобразуем тип данных string в тип даты

df_book = df_book.withColumn("publish_date", to_date(df_book["publish_date"], "yyyy-MM-dd"))
df_auth = df_auth.withColumn("birth_date", to_date(df_auth["birth_date"], "yyyy-MM-dd"))


#df_book = df_book.withColumn("price", col("price").cast("float"))

# проверяем изменения типов данных
df_book.printSchema()
df_auth.printSchema()

# объединяем датафреймы

#join_df = df_book.join(df_auth, on="author_id", how="inner")
join_df = df_book.join(df_auth, df_book.author_id == df_auth.auth_id)

#удаляем фактически дублирующийся столбец auth_id
join_df = join_df.drop("auth_id")

join_df.printSchema()
join_df.show()

# 4.1 находим топ-5 авторов, книги которых принесли наибольшую выручку.

df_41 = join_df.groupBy(["author_id", "name"]).agg(sum("price").alias("total_revenue"))
print("Результат 4.1")
df_41.orderBy(sum("price").desc()).limit(5).show()


# 4.2 находим количество книг в каждом жанре.

genre_df = join_df.groupBy("genre").agg(count("book_id").alias("count"))

print("Результат 4.2")
genre_df.orderBy(count("book_id").desc()).show()

# 4.3 считаем среднюю цену книг по каждому автору.

mean_price = join_df.groupBy(["author_id", "name"]).agg(mean("price").alias("average_price"))
#mean_price = join_df.groupBy(["author_id", "name"]).agg(avg("price").alias("average_price"))

print("Результат 4.3")
mean_price.orderBy(col("average_price").desc()).show()

# 4.4 находим книги, опубликованные после 2000 года, и сортируем их по цене.

df_44 = join_df.filter(year(col("publish_date")) >= 2000).orderBy(col("price").desc())

print("Результат 4.4")
df_44.select(["author_id", "book_id", "title", "genre", "price", "publish_date", "name", "birth_date","country"]).show()

root
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: string (nullable = true)
 |-- publish_date: string (nullable = true)

root
 |-- auth_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_date: string (nullable = true)
 |-- country: string (nullable = true)

root
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: string (nullable = true)
 |-- publish_date: date (nullable = true)

root
 |-- auth_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country: string (nullable = true)

root
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: string (nullable = true)
 |